In [1]:
from modeler.modeler import Modeler as m
from database.market import Market
from database.strategy import Strategy
import pandas as pd
from datetime import datetime
from tqdm import tqdm
pd.options.mode.chained_assignment = None

In [2]:
market = Market()
strategy = Strategy()

In [3]:
market.connect()
prices = market.retrieve("prices")
sp5 = market.retrieve("sp500")
market.disconnect()

In [4]:
prices["date"] = pd.to_datetime(prices["date"])
prices["date"]
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [5]:
strategy.connect()
categories = strategy.retrieve("predicted_stock_categories")
strategy.disconnect()
cats = categories["prediction"].unique()
categories["prediction"] = [x if x != None else "None" for x in categories["prediction"]]
categories.columns
categories["prediction"].unique()

array(['None', '100Materials', '100Information Technology',
       '100Consumer Discretionary', '500Communication Services',
       '200Industrials', '100Consumer Staples', '500Industrials',
       '200Consumer Discretionary', '100Financials', '100Health Care',
       '200Information Technology', '500Health Care', '100Industrials',
       '200Health Care', '500Information Technology', '100Real Estate',
       '100Energy', '500Consumer Discretionary',
       '100Communication Services', '100Utilities', '200Real Estate',
       '200Materials', '200Financials', '200Consumer Staples',
       '200Communication Services', '500Real Estate', '200Energy',
       '500Financials', '200Utilities', '500Consumer Staples',
       '500Materials'], dtype=object)

In [6]:
start = 2013
end = 2022
prices = prices[(prices["year"] >= start) & (prices["year"] <= end)]

In [9]:

    for training_year in tqdm(range(1,6)):
        for category_training_year in tqdm(range(1,5)):
            subset_categories = categories[categories["training_years"]==category_training_year]
            for year in range(2018,2022):
                for quarter in range(1,5):
                    quarterly_categories = subset_categories[(subset_categories["year"]==year) & (subset_categories["quarter"]==quarter)]
                    for category in quarterly_categories["prediction"].unique():
                        try:
                            category_tickers = quarterly_categories[quarterly_categories["prediction"]==category]["ticker"].unique()
                            model_data = data[(data["ticker"].isin(category_tickers))]
                            model_data.sort_values("date",ascending=True,inplace=True)
                            first_index = model_data[(model_data["year"] == year - training_year -1) & (model_data["quarter"]==quarter)].index.values.tolist()[0]
                            last_index = model_data[(model_data["year"] == year) & (model_data["quarter"]==quarter)].index.values.tolist()[0]
                            training_data = model_data.iloc[first_index:last_index]
                            prediction_data = model_data[(model_data["year"] == year) & (model_data["quarter"]==quarter)]
                            training_data.dropna()
                            if training_data.index.size > training_year * 4:
                                X = training_data[[str(x) for x in range(number_of_training_weeks)]]
                                y = training_data["y"]
                                xgb_models = m.xgb_regression({"X":X,"y":y})
#                                 sk_models = m.sk_regression({"X":X,"y":y})
#                                 sk_models.append(xgb_models)
#                                 models = pd.DataFrame(sk_models)
#                                 model = models.sort_values("score",ascending=False).iloc[0]
                                model = xgb_models
                                sim = prediction_data
                                sim["weekly_price_regression_prediction"] = model["model"].predict(sim[[str(x) for x in range(number_of_training_weeks)]])
                                sim["score"] = model["score"]
                                sim["cty"] = category_training_year
                                sim["tw"] = number_of_training_weeks
                                sim["mty"] = training_year
                                sim = sim[["year","quarter","week","ticker","weekly_price_regression_prediction","score","cty","mty","tw"]]
                                strategy.connect()
                                strategy.store("weekly_sim",sim)
                                strategy.disconnect()
                            else:
                                continue
                        except Exception as e:
                            print(year,quarter,category,str(e)) 

  0%|                                                                                                                                             | 0/5 [00:00<?, ?it/s]

  0%|                                                                                                                                             | 0/4 [00:00<?, ?it/s]

2019 4 200Materials list index out of range
2020 3 100Materials list index out of range




 25%|█████████████████████████████████                                                                                                   | 1/4 [08:11<24:34, 491.61s/it]

2021 4 None list index out of range
2021 4 200Information Technology list index out of range
2021 4 100Information Technology list index out of range
2021 4 100Industrials list index out of range
2021 4 200Health Care list index out of range
2021 4 100Consumer Staples list index out of range
2021 4 500Financials list index out of range
2021 4 500Information Technology list index out of range
2021 4 100Consumer Discretionary list index out of range
2021 4 100Health Care list index out of range
2021 4 100Financials list index out of range
2021 4 200Industrials list index out of range
2021 4 100Materials list index out of range
2021 4 100Utilities list index out of range
2021 4 500Health Care list index out of range
2021 4 200Materials list index out of range
2021 4 100Energy list index out of range
2021 4 200Financials list index out of range
2021 4 500Industrials list index out of range
2020 3 200Financials list index out of range




 50%|██████████████████████████████████████████████████████████████████                                                                  | 2/4 [17:19<17:29, 524.50s/it]

...

2021 4 None list index out of range
2021 4 100Financials list index out of range
2021 4 100Health Care list index out of range
2021 4 200Financials list index out of range
2021 4 500Health Care list index out of range
2021 4 100Industrials list index out of range
2021 4 500Information Technology list index out of range
2021 4 200Information Technology list index out of range
2021 4 200Consumer Discretionary list index out of range
2021 4 200Industrials list index out of range
2021 4 100Consumer Discretionary list index out of range
2021 4 200Health Care list index out of range
2021 4 500Industrials list index out of range
2021 4 100Materials list index out of range
2021 4 100Consumer Staples list index out of range
2021 4 100Utilities list index out of range
2021 4 200Materials list index out of range
2021 4 100Real Estate list index out of range
2021 4 500Materials list index out of range
2021 4 100Information Technology list index out of range
2018 1 None list index out of range
2018



 75%|██████████████████████████████████████████████████████████████████████████████████████▎                            | 3/4 [34:06<11:35, 695.01s/it]

2021 4 None list index out of range
2021 4 100Consumer Discretionary list index out of range
2021 4 200Consumer Staples list index out of range
2021 4 100Health Care list index out of range
2021 4 200Health Care list index out of range
2021 4 200Industrials list index out of range
2021 4 100Communication Services list index out of range
2021 4 100Financials list index out of range
2021 4 500Information Technology list index out of range
2021 4 100Materials list index out of range
2021 4 100Utilities list index out of range
2021 4 200Information Technology list index out of range
2021 4 200Real Estate list index out of range
2021 4 500Consumer Discretionary list index out of range
2021 4 100Information Technology list index out of range
2021 4 100Industrials list index out of range
2021 4 500Financials list index out of range
2021 4 100Consumer Staples list index out of range
2021 4 100Energy list index out of range
2021 4 200Consumer Discretionary list index out of range
2021 4 500Heal



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [46:11<00:00, 692.77s/it]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [11:36:53<00:00, 13937.93s/it]

2021 4 None list index out of range
2021 4 100Industrials list index out of range
2021 4 500Health Care list index out of range
2021 4 200Consumer Staples list index out of range
2021 4 200Information Technology list index out of range
2021 4 500Information Technology list index out of range
2021 4 100Health Care list index out of range
2021 4 100Consumer Discretionary list index out of range
2021 4 100Utilities list index out of range
2021 4 100Energy list index out of range
2021 4 200Health Care list index out of range
2021 4 200Financials list index out of range
2021 4 200Consumer Discretionary list index out of range
2021 4 100Communication Services list index out of range
2021 4 100Information Technology list index out of range
2021 4 200Materials list index out of range
2021 4 100Financials list index out of range
2021 4 100Consumer Staples list index out of range
2021 4 100Materials list index out of range


In [ ]:
strategy.connect()
weekly_sim = strategy.retrieve("weekly_sim")
strategy.disconnect()
for number_of_training_weeks in tqdm(range(14,69,14)):
    for weekly_model_training_year in tqdm(range(1,6)):
        for weekly_category_training_year in tqdm(range(1,5)):
            current_weekly_sim = weekly_sim[(weekly_sim["mty"]==weekly_model_training_year) & \
                                           (weekly_sim["tw"]==number_of_training_weeks) & \
                                           (weekly_sim["cty"]==weekly_category_training_year)]
            print(number_of_training_weeks,weekly_model_training_year,weekly_category_training_year,current_weekly_sim.index.size)